In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

color = sns.color_palette('RdBu_r')

# Leitura dos dados
# Inserção de tratamento de exceção para controle de erros
try:
    path = '/home/akel/PycharmProjects/I2A2/data/raw/'
    file = 'desafio6_lixo_eletronico.csv'
    dfo = pd.read_csv(path + file)
    print("✅ Arquivo carregado com sucesso!")

except FileNotFoundError:
    print("❌ Erro: Arquivo não encontrado. Verifique o caminho e tente novamente.")
except pd.errors.EmptyDataError:
    print("❌ Erro: O arquivo está vazio.")
except pd.errors.ParserError:
    print("❌ Erro: Problema ao analisar o arquivo CSV. Verifique o formato.")
except Exception as e:
    print(f"❌ Erro inesperado ao carregar o arquivo: {e}")


✅ Arquivo carregado com sucesso!


In [21]:
def tratar_ids_duplicados(df, coluna_id='ID_Registro'):
    """
    Verifica IDs duplicados e trata conforme as regras:
    - Se dados duplicados: exclui linha
    - Se dados diferentes: gera novo ID inteiro único
    """
    # Fazer cópia do dataframe para não modificar o original
    df_tratado = df.copy()
    
    # Identificar IDs duplicados
    ids_duplicados = df_tratado[df_tratado.duplicated(subset=[coluna_id], keep=False)]
    
    if ids_duplicados.empty:
        print("✅ Nenhum ID duplicado encontrado")
        return df_tratado
    
    print(f"⚠️  Encontrados {len(ids_duplicados)} registros com IDs duplicados")
    
    # Encontrar o maior ID atual para gerar novos IDs acima dele
    max_id = df_tratado[coluna_id].max()
    novos_ids_gerados = 0
    linhas_excluidas = 0
    
    # Lista de todos os IDs existentes (para garantir unicidade)
    ids_existentes = set(df_tratado[coluna_id].tolist())
    
    # Processar cada grupo de IDs duplicados
    for id_valor, grupo in ids_duplicados.groupby(coluna_id):
        if len(grupo) == 1:
            continue  # Apenas um registro, não precisa tratar
            
        print(f"\nProcessando ID {id_valor}: {len(grupo)} registros")
        
        # Verificar se todos os registros são completamente duplicados
        colunas_verificar = [col for col in grupo.columns if col != coluna_id]
        duplicados_completos = grupo.duplicated(subset=colunas_verificar, keep='first')
        
        # Identificar índices para exclusão e modificação
        indices_para_excluir = []
        indices_para_modificar = []
        
        for idx, is_duplicado in duplicados_completos.items():
            if is_duplicado:
                indices_para_excluir.append(idx)
            else:
                indices_para_modificar.append(idx)
        
        # Excluir registros duplicados (mantém apenas o primeiro de cada grupo duplicado)
        if indices_para_excluir:
            df_tratado = df_tratado.drop(indices_para_excluir)
            linhas_excluidas += len(indices_para_excluir)
            print(f"   ➖ Excluídas {len(indices_para_excluir)} linhas duplicadas")
        
        # Gerar novos IDs para registros com dados diferentes
        if indices_para_modificar:
            # Pular o primeiro registro (mantém o ID original)
            indices_para_modificar = indices_para_modificar[1:] if indices_para_modificar else []
            
            for idx in indices_para_modificar:
                # Gerar novo ID inteiro único
                novo_id = max_id + 1
                while novo_id in ids_existentes:
                    novo_id += 1
                
                # Atualizar o ID
                df_tratado.loc[idx, coluna_id] = novo_id
                ids_existentes.add(novo_id)
                max_id = max(max_id, novo_id)
                novos_ids_gerados += 1
                
                print(f"   🔄 Linha {idx}: ID modificado de {id_valor} para {novo_id}")
    return df_tratado

df = tratar_ids_duplicados(dfo, 'ID_Registro')


# Identificar e exibir registros duplicados baseados no ID_Registro
duplicados = df[df.duplicated(subset=['ID_Registro'], keep=False)]

print("=" * 80)
print("🔍 REGISTROS DUPLICADOS - ID_Registro")
print("=" * 80)

if len(duplicados) > 0:
    print(f"Total de registros duplicados: {len(duplicados)}")
    print(f"IDs únicos duplicados: {duplicados['ID_Registro'].nunique()}")
    print("\n📋 Registros duplicados:")
    print(display(duplicados.sort_values('ID_Registro')))
else:
    print("✅ Nenhum registro duplicado encontrado na coluna ID_Registro")


⚠️  Encontrados 10 registros com IDs duplicados

Processando ID 7: 2 registros
   🔄 Linha 14: ID modificado de 7 para 150

Processando ID 22: 2 registros
   🔄 Linha 44: ID modificado de 22 para 151

Processando ID 37: 2 registros
   🔄 Linha 74: ID modificado de 37 para 152

Processando ID 52: 2 registros
   🔄 Linha 104: ID modificado de 52 para 153

Processando ID 67: 2 registros
   🔄 Linha 134: ID modificado de 67 para 154
🔍 REGISTROS DUPLICADOS - ID_Registro
✅ Nenhum registro duplicado encontrado na coluna ID_Registro
